In [0]:
%sql
SELECT * FROM workspace.sc_silver.deals

In [0]:
%sql
SELECT * FROM workspace.sc_silver.leads_pbs

In [0]:
%sql
SELECT * FROM workspace.sc_silver.campaigns

In [0]:
%sql
SELECT * FROM workspace.sc_silver.contactos_pbs

In [0]:
%sql
SELECT COUNT(*) AS num_duplicate_ids
FROM (
  SELECT id
  FROM workspace.sc_silver.contactos_pbs
  GROUP BY id
  HAVING COUNT(*) > 1
);


In [0]:
%sql
SELECT COUNT(*) AS num_duplicate_converted_contacts
FROM (
  SELECT converted_contact
  FROM workspace.sc_silver.leads_pbs
  WHERE converted_contact IS NOT NULL
  GROUP BY converted_contact
  HAVING COUNT(*) > 1
);


In [0]:
%sql
SELECT COUNT(*) AS total_rows
FROM workspace.sc_silver.leads_pbs;


In [0]:
%sql
SELECT COUNT(id) AS non_null_ids
FROM workspace.sc_silver.contactos_pbs;


In [0]:
%sql
SELECT COUNT(DISTINCT c.id) AS common_unique_ids
FROM workspace.sc_silver.contactos_pbs c
JOIN workspace.sc_silver.leads_pbs l
  ON c.id = l.converted_contact;


In [0]:
%sql
SELECT COUNT(DISTINCT leads_pbs.converted_contact) AS common_ids_between_leads_and_campaigns
FROM workspace.sc_silver.leads_pbs
JOIN workspace.sc_silver.campaigns
  ON leads_pbs.converted_contact = campaigns.id
WHERE leads_pbs.converted_contact IS NOT NULL;


In [0]:
%sql
-- Check which campaign IDs exist in leads.id
SELECT DISTINCT id AS campaign_id_in_leads_id
FROM workspace.sc_silver.campaigns
WHERE id IN (
  SELECT id FROM workspace.sc_silver.leads_pbs
);


In [0]:
%sql
SELECT
  campaigns.id AS campaign_id,
  CASE WHEN leads_by_id.id IS NOT NULL THEN 'Yes' ELSE 'No' END AS found_in_leads_id,
  CASE WHEN leads_by_owner.lead_owner IS NOT NULL THEN 'Yes' ELSE 'No' END AS found_in_lead_owner
FROM workspace.sc_silver.campaigns

LEFT JOIN workspace.sc_silver.leads_pbs AS leads_by_id
  ON campaigns.id = leads_by_id.id

LEFT JOIN workspace.sc_silver.leads_pbs AS leads_by_owner
  ON campaigns.id = leads_by_owner.lead_owner;


In [0]:
%sql
SELECT
  campaigns.id AS campaign_id,
  
  -- Lookup flags
  CASE WHEN leads_by_id.id IS NOT NULL THEN 'Yes' ELSE 'No' END AS found_in_leads_id,
  CASE WHEN leads_by_owner.lead_owner IS NOT NULL THEN 'Yes' ELSE 'No' END AS found_in_lead_owner,

  -- Add columns from leads_pbs
  leads_by_id.id AS id_lead,
  leads_by_id.converted_contact

FROM workspace.sc_silver.campaigns

-- Lookup campaigns.id in leads_pbs.id
LEFT JOIN workspace.sc_silver.leads_pbs AS leads_by_id
  ON campaigns.id = leads_by_id.id

-- Lookup campaigns.id in leads_pbs.lead_owner
LEFT JOIN workspace.sc_silver.leads_pbs AS leads_by_owner
  ON campaigns.id = leads_by_owner.lead_owner;


In [0]:
%sql
SELECT
  c.id AS contacto_id,

  -- Flags de correspondência
  CASE WHEN p1.id IS NOT NULL THEN 'Yes' ELSE 'No' END AS match_with_proposta_id,
  CASE WHEN p2.proposta_realizada_owner IS NOT NULL THEN 'Yes' ELSE 'No' END AS match_with_proposta_owner,
  CASE WHEN p3.id_contacto IS NOT NULL THEN 'Yes' ELSE 'No' END AS match_with_id_contacto

FROM workspace.sc_silver.contactos_pbs c

-- Join com propostas_realizadas.id
LEFT JOIN workspace.sc_silver.propostas_realizadas p1
  ON c.id = p1.id

-- Join com propostas_realizadas.proposta_realizada_owner
LEFT JOIN workspace.sc_silver.propostas_realizadas p2
  ON c.id = p2.proposta_realizada_owner

-- Join com propostas_realizadas.id_contacto
LEFT JOIN workspace.sc_silver.propostas_realizadas p3
  ON c.id = p3.id_contacto;


In [0]:
%sql
SELECT COUNT(DISTINCT p3.id_contacto) AS matching_contact_ids
FROM workspace.sc_silver.propostas_realizadas p3
JOIN workspace.sc_silver.contactos_pbs c
    ON p3.id_contacto = c.id
WHERE p3.id_contacto IS NOT NULL;


In [0]:
%sql
SELECT COUNT(DISTINCT p.id_contacto) AS matching_contact_ids
FROM workspace.sc_silver.propostas_realizadas p
JOIN workspace.sc_silver.deals d
  ON p.id_contacto = d.id_contacto
WHERE p.id_contacto IS NOT NULL;


In [0]:
%sql
SELECT COUNT(*) AS total_rows
FROM workspace.sc_silver.leads_pbs;


In [0]:
%sql
SELECT
  l.converted_contact,
  CASE WHEN d1.id IS NOT NULL THEN 'Yes' ELSE 'No' END AS match_in_deals_id,
  CASE WHEN d2.converted_from_lead IS NOT NULL THEN 'Yes' ELSE 'No' END AS match_in_deals_converted_from_lead
FROM workspace.sc_silver.leads_pbs l
LEFT JOIN workspace.sc_silver.deals d1
  ON l.converted_contact = d1.id
LEFT JOIN workspace.sc_silver.deals d2
  ON l.converted_contact = d2.converted_from_lead
WHERE l.converted_contact IS NOT NULL;


In [0]:
%sql
-- How many leads match deals.id
SELECT COUNT(DISTINCT l.id) AS leads_in_deals_id
FROM workspace.sc_silver.leads_pbs l
JOIN workspace.sc_silver.deals d ON l.id = d.id;

-- How many leads match deals.converted_from_lead
SELECT COUNT(DISTINCT l.id) AS leads_in_converted_from_lead
FROM workspace.sc_silver.leads_pbs l
JOIN workspace.sc_silver.deals d ON l.id = d.converted_from_lead;


In [0]:
%sql
SELECT COUNT(DISTINCT d.campanha) AS matched_campaigns_in_deals
FROM workspace.sc_silver.deals d
JOIN workspace.sc_silver.campaigns c
  ON d.campanha = c.id
WHERE d.campanha IS NOT NULL;


In [0]:
%sql
SELECT COUNT(*) AS num_duplicate_converted_contacts
FROM (
  SELECT converted_contact
  FROM workspace.sc_silver.leads_pbs
  WHERE converted_contact IS NOT NULL
  GROUP BY converted_contact
  HAVING COUNT(*) > 1
);


In [0]:
%sql
SELECT
  occurrences,
  COUNT(*) AS num_contacts_with_this_occurrence
FROM (
  SELECT converted_contact, COUNT(*) AS occurrences
  FROM workspace.sc_silver.leads_pbs
  WHERE converted_contact IS NOT NULL
  GROUP BY converted_contact
) AS sub
GROUP BY occurrences
ORDER BY occurrences;


In [0]:
%sql
SELECT COUNT(*) AS num_duplicate_ids
FROM (
  SELECT id
  FROM workspace.sc_silver.campaigns
  GROUP BY id
  HAVING COUNT(*) > 1
);


In [0]:
%sql
SELECT 
  id_contacto, 
  COUNT(*) AS occurrences
FROM workspace.sc_silver.deals
WHERE id_contacto IS NOT NULL
GROUP BY id_contacto
HAVING COUNT(*) > 1
ORDER BY occurrences DESC;


In [0]:
%sql
/*********************************************
SELECT
  COUNT(*) AS total_rows,
  COUNT(CASE WHEN apoio_concessionario IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_apoio_concessionario,
  COUNT(CASE WHEN apoio_total IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_apoio_total,
  COUNT(CASE WHEN sub_total_com_extras IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_sub_total_com_extras,
  COUNT(CASE WHEN descricao_do_pedido_de_apoio IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_descricao_do_pedido_de_apoio,
  COUNT(CASE WHEN estado_do_pedido IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_estado_do_pedido,
  COUNT(CASE WHEN apoio_percentual IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_apoio_percentual,
  COUNT(CASE WHEN data_de_criacao_da_proposta IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_de_criacao_da_proposta,
  COUNT(CASE WHEN valor_campanhas_comerciais IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_valor_campanhas_comerciais,
  COUNT(CASE WHEN ofertas_de_campanha IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_ofertas_de_campanha,
  COUNT(CASE WHEN resposta_do_importador IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_resposta_do_importador,
  COUNT(CASE WHEN data_de_entrega_da_proposta IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_de_entrega_da_proposta,
  COUNT(CASE WHEN desconto_total__c__apoio_de_importador_ IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_desconto_total__c__apoio_de_importador_,
  COUNT(CASE WHEN codigo_cor_exterior IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_codigo_cor_exterior,
  COUNT(CASE WHEN codigo_cor_interior IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_codigo_cor_interior,
  COUNT(CASE WHEN concessionario_owner IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_concessionario_owner,
  COUNT(CASE WHEN valor_aprovado IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_valor_aprovado,
  COUNT(CASE WHEN estado_do_contrato IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_estado_do_contrato,
  COUNT(CASE WHEN data_prevista_matricula IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_prevista_matricula,
  COUNT(CASE WHEN data_prevista_de_entrega IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_prevista_de_entrega,
  COUNT(CASE WHEN data_da_conclusao IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_data_da_conclusao,
  COUNT(CASE WHEN id_classe IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_id_classe,
  COUNT(CASE WHEN descricao_classe IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_descricao_classe,
  COUNT(CASE WHEN id_model_group IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_id_model_group,
  COUNT(CASE WHEN descricao_model_group IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_descricao_model_group,
  COUNT(CASE WHEN valid_until IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_valid_until,
  COUNT(CASE WHEN forma_de_pagamento IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_forma_de_pagamento,
  COUNT(CASE WHEN nome_da_campanha IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_nome_da_campanha,
  COUNT(CASE WHEN _fivetran_index IS NULL THEN 1 END) * 100.0 / COUNT(*) AS pct_null_fivetran_index
FROM workspace.sc_silver.propostas_realizadas;
*********************************************/

In [0]:
from pyspark.sql import functions as F

# ==== DEALS ====
df_deals = spark.table("workspace.sc_silver.deals")
df_deals_renamed = df_deals.select(
    *[F.col(c).alias(f"{c}_deals") for c in df_deals.columns]
)
df_deals_renamed.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.deals_renamed")
display(spark.table("workspace.sc_silver.deals_renamed"))

# ==== CAMPAIGNS ====
df_campaigns = spark.table("workspace.sc_silver.campaigns")
df_campaigns_renamed = df_campaigns.select(
    *[F.col(c).alias(f"{c}_campaigns") for c in df_campaigns.columns]
)
df_campaigns_renamed.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.campaigns_renamed")
display(spark.table("workspace.sc_silver.campaigns_renamed"))


In [0]:
from pyspark.sql import functions as F

# Load renamed tables
df_deals_renamed = spark.table("workspace.sc_silver.deals_renamed")
df_campaigns_renamed = spark.table("workspace.sc_silver.campaigns_renamed")

# Join on campanha_deals = id_campaigns
df_deals_with_campaigns = df_deals_renamed.join(
    df_campaigns_renamed,
    df_deals_renamed.campanha_deals == df_campaigns_renamed.id_campaigns,
    "left"
)

# Save as permanent table
df_deals_with_campaigns.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.deals_with_campaigns")

# Preview
display(spark.table("workspace.sc_silver.deals_with_campaigns"))


In [0]:
from pyspark.sql import functions as F

# Load original table
df = spark.table("workspace.sc_silver.propostas_realizadas")

# Build SELECT list with aliases (append _propostas_realizadas to each column)
select_expr = [F.col(c).alias(f"{c}_propostas_realizadas") for c in df.columns]
df_renamed = df.select(*select_expr)

# Write to a NEW table so you don't overwrite original
df_renamed.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.propostas_realizadas_renamed")

# Display the new table
display(spark.table("workspace.sc_silver.propostas_realizadas_renamed"))

In [0]:
#  JOIN propostas_realizadas_renamed with deals_with_campaigns ====

# Load both tables
df_propostas_renamed = spark.table("workspace.sc_silver.propostas_realizadas_renamed")
df_deals_with_campaigns = spark.table("workspace.sc_silver.deals_with_campaigns")

# LEFT join from propostas to deals
df_joined = df_propostas_renamed.join(
    df_deals_with_campaigns,
    df_propostas_renamed.id_contacto_propostas_realizadas == df_deals_with_campaigns.id_contacto_deals,
    "left"
)

# Save as propostas_with_deals_with_campaigns
df_joined.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.propostas_with_deals_with_campaigns")

# Display the result
display(spark.table("workspace.sc_silver.propostas_with_deals_with_campaigns"))

In [0]:
%sql
SELECT *
FROM workspace.sc_silver.leads_pbs
LIMIT 100;


In [0]:
%sql
SELECT *
FROM workspace.sc_silver.contactos_pbs
LIMIT 100;


In [0]:
%sql
SELECT COUNT(DISTINCT c.id) AS common_unique_ids
FROM workspace.sc_silver.contactos_pbs c
JOIN workspace.sc_silver.leads_pbs l
  ON c.id = l.converted_contact;


In [0]:
%sql
/*********************************************
SELECT 
  COUNT(DISTINCT c.id) AS common_unique_ids,
  COUNT(DISTINCT CASE 
    WHEN c.data_criacao_da_lead = l.created_time THEN c.id
  END) AS common_unique_ids_with_date
FROM workspace.sc_silver.contactos_pbs c
JOIN workspace.sc_silver.leads_pbs l
  ON c.id = l.converted_contact;
  *********************************************/


In [0]:
from pyspark.sql import functions as F

# ==== LEADS ====
df_leads = spark.table("workspace.sc_silver.leads_pbs")
df_leads_renamed = df_leads.select(
    *[F.col(c).alias(f"{c}_leads") for c in df_leads.columns]
)
df_leads_renamed.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.leads_renamed")
display(spark.table("workspace.sc_silver.leads_renamed"))

# ==== CONTACTOS ====
df_contactos = spark.table("workspace.sc_silver.contactos_pbs")
df_contactos_renamed = df_contactos.select(
    *[F.col(c).alias(f"{c}_contactos") for c in df_contactos.columns]
)
df_contactos_renamed.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.contactos_renamed")
display(spark.table("workspace.sc_silver.contactos_renamed"))


In [0]:
from pyspark.sql import functions as F

# Load both renamed tables
df_leads = spark.table("workspace.sc_silver.leads_renamed")
df_contactos = spark.table("workspace.sc_silver.contactos_renamed")

# LEFT join on both ID and date
df_joined = df_leads.join(
    df_contactos,
    (df_leads.converted_contact_leads == df_contactos.id_contactos) &
    (F.to_timestamp(df_leads.created_time_leads, "dd-MM-yyyy HH:mm") ==
     F.to_timestamp(df_contactos.data_criacao_da_lead_contactos, "dd-MM-yyyy HH:mm")),
    "left"
)

# Save as a new permanent table
df_joined.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.leads_with_contactos")

# Display preview
display(spark.table("workspace.sc_silver.leads_with_contactos"))


In [0]:
from pyspark.sql import functions as F

# Load the new table
df_new = spark.table("workspace.sc_silver.leads_with_contactos")

# Calculate the counts
df_counts = df_new.agg(
    F.countDistinct("id_contactos").alias("common_unique_ids"),
    F.countDistinct(
        F.when(
            F.to_timestamp(df_new.data_criacao_da_lead_contactos, "dd-MM-yyyy HH:mm") ==
            F.to_timestamp(df_new.created_time_leads, "dd-MM-yyyy HH:mm"),
            df_new.id_contactos
        )
    ).alias("common_unique_ids_with_date")
)

df_counts.show()


In [0]:
%sql

SELECT *
FROM workspace.sc_silver.propostas_with_deals_with_campaigns
LIMIT 100;


In [0]:
%sql
-- First table
SELECT *
FROM workspace.sc_silver.leads_with_contactos
LIMIT 100;


In [0]:
from pyspark.sql import functions as F

# Load both tables
df_leads_contactos = spark.table("workspace.sc_silver.leads_with_contactos")
df_propostas = spark.table("workspace.sc_silver.propostas_with_deals_with_campaigns")

# LEFT join on id_contactos vs id_contacto_propostas_realizadas
df_funil = df_leads_contactos.join(
    df_propostas,
    df_leads_contactos.id_contactos == df_propostas.id_contacto_propostas_realizadas,
    "left"
)

# Save as permanent table
df_funil.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.sc_silver.Funil")

# Preview
display(spark.table("workspace.sc_silver.Funil"))


In [0]:
%sql
SHOW TABLES IN workspace.sc_silver;

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StringType, DoubleType, FloatType
)

# Load table
df = spark.table("sc_silver.funil")

total_rows = df.count()
if total_rows == 0:
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs = []
    col_names = []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    tidy.orderBy(F.desc("blank_pct")).show(truncate=False)


In [0]:
tidy.orderBy(F.desc("blank_pct")).show(n=tidy.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, FloatType

tables = [
    "bd_rede_hyundai",
    "campanha_vouchers",
    "campanhas_tecnicas",
    "contactos_importador_pbs",
    "contratos_financiamento",
    "historico_de_servicos",
    "viaturas",
    "viaturas_demo"
]

def blank_percentage_for_table(table_name):
    print(f"=== Processing table: {table_name} ===")
    df = spark.table(f"sc_silver.{table_name}")
    total_rows = df.count()

    if total_rows == 0:
        print(f"Table {table_name} is empty.")
        return spark.createDataFrame([], "column string, blank_pct double")

    agg_exprs = []
    col_names = []

    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    pct_row = df.agg(*agg_exprs)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    return tidy.orderBy(F.desc("blank_pct"))

# Loop through all tables and show results
for tbl in tables:
    result_df = blank_percentage_for_table(tbl)
    result_df.show(n=result_df.count(), truncate=False)


In [0]:
%sql
SELECT * FROM sc_silver.bd_rede_hyundai;

In [0]:
%sql
SELECT * FROM sc_silver.campanha_vouchers;

In [0]:
%sql
SELECT * FROM sc_silver.campanhas_tecnicas;

In [0]:
%sql
SELECT * FROM sc_silver.contactos_importador_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contactos_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contas_importador_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contas_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contratos_financiamento;

In [0]:
%sql
SELECT * FROM sc_silver.deals;

In [0]:
%sql
SELECT * FROM sc_silver.historico_de_servicos;

In [0]:
%sql
SELECT * FROM sc_silver.leads_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.propostas_realizadas;

In [0]:
%sql
SELECT * FROM sc_silver.viaturas;

In [0]:
%sql
SELECT * FROM sc_silver.viaturas_demo;

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StringType, DoubleType, FloatType
)

# Load table
df = spark.table("sc_silver.bd_rede_hyundai")

total_rows = df.count()
if total_rows == 0:
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs = []
    col_names = []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    tidy.orderBy(F.desc("blank_pct")).show(truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StringType, DoubleType, FloatType
)

# Load table
df = spark.table("sc_silver.campaigns")

total_rows = df.count()
if total_rows == 0:
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs = []
    col_names = []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    tidy.orderBy(F.desc("blank_pct")).show(truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StringType, DoubleType, FloatType
)

# Load table
df = spark.table("sc_silver.campanha_vouchers")

total_rows = df.count()
if total_rows == 0:
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs = []
    col_names = []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    tidy.orderBy(F.desc("blank_pct")).show(truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StringType, DoubleType, FloatType
)

# Load table
df = spark.table("sc_silver.campanhas_tecnicas")

total_rows = df.count()
if total_rows == 0:
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs = []
    col_names = []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    tidy.orderBy(F.desc("blank_pct")).show(truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, FloatType

# ---------- Config ----------
table_name = "sc_silver.contactos_importador_pbs"  # <- change if needed
only_above_pct = None  # e.g. set to 0 or 50 to filter; use None to show ALL
# ----------------------------

df = spark.table(table_name)

total_rows = df.count()
if total_rows == 0:
    print(f"{table_name} is empty.")
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs, col_names = [], []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    # Optional filter
    if only_above_pct is not None:
        tidy = tidy.filter(F.col("blank_pct") > float(only_above_pct))

    tidy = tidy.orderBy(F.desc("blank_pct"))

    # >>> Show ALL rows
    tidy.show(tidy.count(), truncate=False)


In [0]:
'''
spark.table("sc_silver.contactos_importador_pbs") \
    .select("pais") \
    .distinct() \
    .show(truncate=False)
'''

In [0]:
'''
spark.table("sc_silver.contactos_importador_pbs") \
    .groupBy("pais") \
    .count() \
    .orderBy(F.desc("count")) \
    .show(truncate=False)
'''

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, FloatType

# ---------- Config ----------
table_name = "sc_silver.contactos_pbs"  # <- changed to contactos_pbs
only_above_pct = None  # e.g. set to 0 or 50 to filter; use None to show ALL
# ----------------------------

df = spark.table(table_name)

total_rows = df.count()
if total_rows == 0:
    print(f"{table_name} is empty.")
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs, col_names = [], []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    # Optional filter
    if only_above_pct is not None:
        tidy = tidy.filter(F.col("blank_pct") > float(only_above_pct))

    tidy = tidy.orderBy(F.desc("blank_pct"))

    # >>> Show ALL rows
    tidy.show(tidy.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, FloatType

# ---------- Config ----------
table_name = "sc_silver.contas_importador_pbs"  # <- changed to contas_importador_pbs
only_above_pct = None  # e.g. set to 0 or 50 to filter; use None to show ALL
# ----------------------------

df = spark.table(table_name)

total_rows = df.count()
if total_rows == 0:
    print(f"{table_name} is empty.")
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs, col_names = [], []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    # Optional filter
    if only_above_pct is not None:
        tidy = tidy.filter(F.col("blank_pct") > float(only_above_pct))

    tidy = tidy.orderBy(F.desc("blank_pct"))

    # >>> Show ALL rows
    tidy.show(tidy.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, FloatType

# ---------- Config ----------
table_name = "sc_silver.contas_pbs"  # <- changed to contas_pbs
only_above_pct = None  # e.g. set to 0 or 50 to filter; use None to show ALL
# ----------------------------

df = spark.table(table_name)

total_rows = df.count()
if total_rows == 0:
    print(f"{table_name} is empty.")
    spark.createDataFrame([], "column string, blank_pct double").show()
else:
    # Build one aggregation per column
    agg_exprs, col_names = [], []
    for field in df.schema.fields:
        c = field.name
        dt = field.dataType

        if isinstance(dt, StringType):
            # NULL or empty/whitespace
            cond = F.col(c).isNull() | (F.length(F.trim(F.col(c))) == 0)
        elif isinstance(dt, (DoubleType, FloatType)):
            # NULL or NaN
            cond = F.col(c).isNull() | F.isnan(F.col(c))
        else:
            # Other types: only NULL counts as blank
            cond = F.col(c).isNull()

        agg_exprs.append(F.round(F.avg(F.when(cond, 1).otherwise(0)) * 100, 4).alias(c))
        col_names.append(c)

    # One row with % per column
    pct_row = df.agg(*agg_exprs)

    # Reshape to (column, blank_pct)
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in col_names])
    tidy = pct_row.selectExpr(f"stack({len(col_names)}, {stack_expr}) as (column, blank_pct)")

    # Optional filter
    if only_above_pct is not None:
        tidy = tidy.filter(F.col("blank_pct") > float(only_above_pct))

    tidy = tidy.orderBy(F.desc("blank_pct"))

    # >>> Show ALL rows
    tidy.show(tidy.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

table = "sc_silver.contas_pbs"
cols = ["tasks_involved", "events_involved", "calls_involved", "rent_a_car"]

df = spark.table(table)
total = df.count()

# Only keep columns that actually exist (avoid errors)
present = [c for c in cols if c in df.columns]
missing = [c for c in cols if c not in df.columns]
if missing:
    print("Skipping missing columns:", ", ".join(missing))

for c in present:
    print(f"\n=== {c} ===")
    # Treat NULLs explicitly so they show up in counts
    out = (
        df.groupBy(F.coalesce(F.col(c).cast("string"), F.lit("<NULL>")).alias(c))
          .count()
          .withColumn("pct", F.round(F.col("count") / F.lit(total) * 100, 2))
          .orderBy(F.desc("count"))
    )
    out.show(out.count(), truncate=False)  # show all rows for this column


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.contratos_financiamento"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [
        F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols
    ]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.deals"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.historico_de_servicos"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

table = "sc_silver.historico_de_servicos"
cols = ["ciclo_de_fidelizacao", "dadosconcessao"]

df = spark.table(table)
total = df.count()

# Only keep columns that actually exist (avoid errors)
present = [c for c in cols if c in df.columns]
missing = [c for c in cols if c not in df.columns]
if missing:
    print("Skipping missing columns:", ", ".join(missing))

for c in present:
    print(f"\n=== {c} ===")
    # Treat NULLs explicitly so they show up in counts
    out = (
        df.groupBy(F.coalesce(F.col(c).cast("string"), F.lit("<NULL>")).alias(c))
          .count()
          .withColumn("pct", F.round(F.col("count") / F.lit(total) * 100, 2))
          .orderBy(F.desc("count"))
    )
    out.show(out.count(), truncate=False)  # show all rows for this column


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.leads_pbs"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.propostas_realizadas"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.viaturas"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.viaturas"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # One pass: compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
from pyspark.sql import functions as F

# Table to profile
table_name = "sc_silver.viaturas_demo"

df = spark.table(table_name)
total_rows = df.count()

if total_rows == 0:
    print(f"{table_name} is empty.")
else:
    cols = df.columns

    # Compute NULL counts for every column
    agg_exprs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in cols]
    null_counts_row = df.agg(*agg_exprs)

    # Reshape to (column, null_count) and add percentage
    stack_expr = ", ".join([f"'{c}', `{c}`" for c in cols])
    out = (
        null_counts_row.selectExpr(f"stack({len(cols)}, {stack_expr}) as (column, null_count)")
        .withColumn("total_rows", F.lit(total_rows))
        .withColumn("null_pct", F.round(F.col("null_count") / F.col("total_rows") * 100, 4))
        .orderBy(F.desc("null_pct"), F.desc("null_count"))
    )

    # Show all rows
    out.show(out.count(), truncate=False)


In [0]:
%sql
SHOW TABLES IN workspace.sc_silver;

In [0]:
%sql
SELECT * FROM sc_silver.contas_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contas_importador_pbs;


In [0]:
%sql
SELECT * FROM sc_silver.campanha_vouchers;

In [0]:
%sql
SELECT * FROM sc_silver.campaigns;

In [0]:
%sql
SELECT * FROM sc_silver.bd_rede_hyundai;

In [0]:
%sql
SELECT * FROM sc_silver.campanhas_tecnicas;

In [0]:
%sql
SELECT * FROM sc_silver.contactos_importador_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contactos_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.contratos_financiamento;

In [0]:
%sql
SELECT * FROM sc_silver.deals;

In [0]:
%sql
SELECT * FROM sc_silver.historico_de_servicos;

In [0]:
%sql
SELECT * FROM sc_silver.leads_pbs;

In [0]:
%sql
SELECT * FROM sc_silver.propostas_realizadas;

In [0]:
%sql
SELECT * FROM sc_silver.viaturas;

In [0]:
%sql

SELECT * FROM sc_silver.viaturas_demo;

In [0]:
%sql
-- 1) See each duplicate pair and how many times it appears
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_silver.leads_pbs
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC;

-- 2) Count total duplicate rows (excluding first occurrence in each group)
WITH grouped AS (
    SELECT id, created_time, COUNT(*) AS cnt
    FROM sc_silver.leads_pbs
    GROUP BY id, created_time
)
SELECT SUM(cnt - 1) AS total_duplicate_rows
FROM grouped
WHERE cnt > 1;


In [0]:
%sql
SELECT *
FROM (
    SELECT *,
           COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
    FROM sc_silver.leads_pbs
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
SELECT a.*
FROM sc_silver.leads_pbs a
JOIN (
  SELECT _fivetran_id, ROW_NUMBER() OVER (PARTITION BY id, created_time ORDER BY _fivetran_id) rn
  FROM sc_silver.leads_pbs
) b
ON a._fivetran_id = b._fivetran_id
WHERE b.rn > 1
ORDER BY a.id, a.created_time;


In [0]:
%sql
-- 1) See each duplicate pair and how many times it appears
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_silver.leads_pbs
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC;

-- 2) Count total duplicate rows (excluding first occurrence in each group)
WITH grouped AS (
    SELECT id, created_time, COUNT(*) AS cnt
    FROM sc_silver.leads_pbs
    GROUP BY id, created_time
)
SELECT SUM(cnt - 1) AS total_duplicate_rows
FROM grouped
WHERE cnt > 1;

In [0]:
%sql
SELECT *
FROM (
    SELECT *,
           COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
    FROM sc_silver.leads_pbs
) t
WHERE cnt > 1
ORDER BY id, created_time;

In [0]:
%sql
-- 1) Duplicate (id, data_criacao_da_lead) pairs and their counts
SELECT id, data_criacao_da_lead, COUNT(*) AS cnt
FROM sc_silver.Contactos_PBS
GROUP BY id, data_criacao_da_lead
HAVING COUNT(*) > 1
ORDER BY cnt DESC;

-- 2) Total number of duplicate rows (excluding the first in each pair)
WITH grouped AS (
  SELECT id, data_criacao_da_lead, COUNT(*) AS cnt
  FROM sc_silver.Contactos_PBS
  GROUP BY id, data_criacao_da_lead
)
SELECT COALESCE(SUM(cnt - 1), 0) AS total_duplicate_rows
FROM grouped
WHERE cnt > 1;


In [0]:
%sql
SELECT *
FROM (
  SELECT *,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contactos_pbs
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- 1) Duplicate (id, created_time) pairs and their counts
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_silver.contactos_pbs
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id, created_time;

-- 2) Total duplicate rows (excluding the first in each pair)
WITH grouped AS (
  SELECT id, created_time, COUNT(*) AS cnt
  FROM sc_silver.contactos_pbs
  GROUP BY id, created_time
)
SELECT COALESCE(SUM(cnt - 1), 0) AS total_duplicate_rows
FROM grouped
WHERE cnt > 1;


In [0]:
%sql
SELECT
  id,
  created_time,
  COUNT(*) AS cnt,
  COLLECT_SET(id_proposta_realizada) AS proposta_ids  -- use COLLECT_LIST if you want repeats
FROM sc_silver.propostas_realizadas
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id, created_time;


In [0]:
%sql
SELECT
  id,
  created_time,
  id_proposta_realizada,
  subject,
  proposta_realizada_owner,
  proposta_realizada_owner_name,
  cnt
FROM (
  SELECT *,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.propostas_realizadas
) t
WHERE cnt > 1
ORDER BY id, created_time, id_proposta_realizada;


In [0]:
%sql
SELECT *
FROM (
  SELECT *,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contactos_pbs
) t
WHERE cnt > 1
ORDER BY id, created_time;

In [0]:
%sql
-- Show the actual rows that belong to duplicated (id, created_time) pairs
SELECT *
FROM (
  SELECT d.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.deals d
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_silver.deals
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id, created_time;


In [0]:
%sql
-- (Optional) Preview rows that would be deleted
SELECT a.*
FROM sc_silver.deals a
JOIN (
  SELECT _fivetran_id,
         ROW_NUMBER() OVER (
           PARTITION BY id, created_time
           ORDER BY _fivetran_id          -- pick your preferred tiebreaker
         ) AS rn
  FROM sc_silver.deals
) r
ON a._fivetran_id = r._fivetran_id
WHERE r.rn > 1
ORDER BY a.id, a.created_time, a._fivetran_id;


In [0]:
%sql
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_silver.deals
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id, created_time;

In [0]:
%sql
-- sc_silver.contas_pbs
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contas_pbs t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.contas_importador_pbs
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contas_importador_pbs t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.campanha_vouchers
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.campanha_vouchers t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.bd_rede_hyundai
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.bd_rede_hyundai t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.campanhas_tecnicas
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.campanhas_tecnicas t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.contactos_importador_pbs
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contactos_importador_pbs t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.contratos_financiamento
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.contratos_financiamento t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.historico_de_servicos
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.historico_de_servicos t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.viaturas
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.viaturas t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
%sql
-- sc_silver.viaturas_demo
SELECT *
FROM (
  SELECT t.*,
         COUNT(*) OVER (PARTITION BY id, created_time) AS cnt
  FROM sc_silver.viaturas_demo t
) t
WHERE cnt > 1
ORDER BY id, created_time;


In [0]:
# Databricks: run as a Python cell
from pyspark.sql import functions as F, types as T

TABLES = [
    "sc_silver.bd_rede_hyundai",
    "sc_silver.campaigns",
    "sc_silver.campanha_vouchers",
    "sc_silver.campanhas_tecnicas",
    "sc_silver.contactos_importador_pbs",
    "sc_silver.contactos_pbs",
    "sc_silver.contas_importador_pbs",
    "sc_silver.contas_pbs",
    "sc_silver.contratos_financiamento",
    "sc_silver.deals",
    "sc_silver.historico_de_servicos",
    "sc_silver.leads_pbs",
    "sc_silver.propostas_realizadas",
    "sc_silver.viaturas",
    "sc_silver.viaturas_demo",
]

def preserve_case(colname: str) -> bool:
    """
    Columns to NOT proper-case (keep original case, only trim):
    IDs, emails, URLs, VIN/IBAN, geo codes, date/time-like, etc.
    """
    c = colname.lower()
    if c == "id" or c.endswith("_id") or c.endswith("id"):
        return True
    sensitive_terms = [
        "email", "mail", "url", "http", "https",
        "vin", "iban", "bic", "swift", "uuid", "guid", "imei", "ip", "mac",
        "nif", "nipc", "niss", "vat",
        "postal", "codigo_postal", "zip", "cep",
        "matricula", "placa", "chassis", "plate",
        "lat", "lng", "lon", "latitude", "longitude",
    ]
    if any(term in c for term in sensitive_terms):
        return True
    # date/time-like names
    if "date" in c or "data" in c or "time" in c or "timestamp" in c:
        return True
    return False

def clean_string_col(name: str):
    col = F.col(name)
    # Trim ONLY ends; DO NOT normalize internal spaces or remove control chars; DO NOT map "" -> NULL
    col = F.trim(col)
    # Proper-case (per word) unless column should preserve case
    if not preserve_case(name):
        col = F.initcap(F.lower(col))  # e.g., "JOÃO DA SILVA" -> "João Da Silva"
    return col.alias(name)

def clean_df(df):
    out = []
    for f in df.schema.fields:
        if isinstance(f.dataType, T.StringType):
            out.append(clean_string_col(f.name))
        else:
            # non-string columns unchanged
            out.append(F.col(f.name))
    return df.select(*out)

for full_name in TABLES:
    df = spark.table(full_name)
    df_clean = clean_df(df)
    # Overwrite original table in place (Delta)
    df_clean.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(full_name)
    print(f"Overwrote: {full_name}")


In [0]:
%sql

SELECT * FROM sc_silver.bd_rede_hyundai;

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.bd_rede_hyundai"
TOP_N = 100        # change or remove the .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Skip complex types that can't be grouped directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)  # remove to see all
         .withColumnRenamed(c, "value")
         .select("value", "count")  # no need to show the column name in the result
    )


In [0]:
%sql
SELECT * FROM sc_silver.campaigns;

In [0]:
%sql
SELECT * FROM sc_silver.campanha_vouchers;

In [0]:
%sql
SELECT * FROM sc_silver.campanhas_tecnicas;

In [0]:
%sql
SELECT * FROM sc_silver.contactos_importador_pbs;

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.contactos_importador_pbs"
TOP_N = 100        # change or remove the .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Skip complex types that can't be grouped directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)                     # remove to see all values
         .withColumnRenamed(c, "value")
         .select("value", "count")         # result has just the distinct values + counts
    )


In [0]:
%sql
WITH cleaned AS (
  SELECT
    tipo_viatura,
    -- base text before the first [, else the first quoted item inside [...]
    trim(
      coalesce(
        regexp_extract(tipo_viatura, '^[^\\[]+', 0),   -- e.g. "Novos["Usados"]" -> "Novos"
        regexp_extract(tipo_viatura, '"([^"]+)"', 1)   -- e.g. '["Novos"]' -> "Novos"
      )
    ) AS base
  FROM sc_silver.contactos_importador_pbs
)
SELECT
  CASE
    WHEN lower(base) rlike '^nov'              THEN 'Novos'
    WHEN lower(base) rlike '^(usad|semi)'      THEN 'Usados'
    WHEN lower(base) rlike '^emp'              THEN 'Empresas'
    WHEN base = '' OR base IS NULL             THEN NULL
    ELSE initcap(base)
  END AS tipo_viatura_clean,
  count(*) AS cnt
FROM cleaned
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT tipo_viatura, count(*) AS cnt
FROM sc_silver.contactos_importador_pbs
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT tipo_viatura, COUNT(*) AS cnt
FROM sc_silver.contactos_importador_pbs
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT tipo_viatura, COUNT(*) AS cnt
FROM sc_silver.contactos_importador_pbs
GROUP BY 1
ORDER BY cnt DESC;

In [0]:
%sql
WITH tok AS (
  SELECT
    *,
    -- keep only digits & commas, then split; drop empties; dedupe
    array_distinct(
      filter(
        split(regexp_replace(coalesce(id_contactos,''), '[^0-9,]', ''), ','),
        x -> x <> ''
      )
    ) AS id_list
  FROM sc_silver.contactos_importador_pbs
)
SELECT size(id_list) AS n_ids, count(*) AS cnt
FROM tok
GROUP BY 1 ORDER BY 1 DESC;


In [0]:
%sql
SELECT * FROM sc_silver.contactos_importador_pbs;

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.contactos_pbs"
TOP_N = 100        # change or remove the .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Skip complex types that can't be grouped directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)                     # remove to see all values
         .withColumnRenamed(c, "value")
         .select("value", "count")         # result has just the distinct values + counts
    )


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.contas_importador_pbs"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.contas_pbs"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.contratos_financiamento"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
SELECT * FROM sc_silver.contratos_financiamento;

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.deals"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
-- Preview the normalization (no write)
WITH parsed AS (
  SELECT *,
    filter(
      array_distinct(array(
        CASE WHEN regexp_like(categoria_negocio, '(?i)nov')                      THEN 'Novos'    END,
        CASE WHEN regexp_like(categoria_negocio, '(?i)usad')                     THEN 'Usados'   END,
        CASE WHEN regexp_like(categoria_negocio, '(?i)demo')                     THEN 'Demo'     END,
        CASE WHEN regexp_like(categoria_negocio, '(?i)servi(ç|c)o?s?')           THEN 'Servicos' END
      )),
      x -> x IS NOT NULL
    ) AS cats
  FROM sc_silver.deals
)
SELECT nullif(array_join(array_sort(cats), ';'), '') AS categoria_norm, count(*) AS cnt
FROM parsed
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
WITH mapped AS (
  SELECT
    CASE
      WHEN regexp_like(categoria_negocio, '(?i)demo')                     THEN 'Demo'
      WHEN regexp_like(categoria_negocio, '(?i)nov')                      THEN 'Novos'
      WHEN regexp_like(categoria_negocio, '(?i)usad')                     THEN 'Usados'
      WHEN regexp_like(categoria_negocio, '(?i)servi(ç|c)o?s?')           THEN NULL
      ELSE NULL
    END AS categoria_final
  FROM sc_silver.deals
)
SELECT categoria_final, COUNT(*) cnt
FROM mapped
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.historico_de_servicos"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
WITH src AS (
  SELECT
    tipo_de_servico AS raw,
    regexp_replace(lower(coalesce(tipo_de_servico,'')), '[^a-z]', '') AS s
  FROM sc_silver.historico_de_servicos
)
SELECT
  CASE
    WHEN s RLIKE 'manut|rev|serv'                 THEN 'Manutenção'
    WHEN s RLIKE 'rep|arranj|cons'                THEN 'Reparação'
    WHEN s RLIKE 'mec|el|motor|electricidade'                 THEN 'Mecânica'
    WHEN s RLIKE 'chapa|chapar|chapari'                THEN 'Chapa'
    WHEN s RLIKE 'pint'                              THEN 'Pintura'
    WHEN s RLIKE 'colis'                               THEN 'Colisão'
ELSE 'Manutenção'
  END AS categoria_norm,
  COUNT(*) AS cnt
FROM src
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT tipo_de_servico, COUNT(*) AS cnt
FROM sc_silver.historico_de_servicos
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.leads_pbs"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
SELECT lead_status_2, COUNT(*) AS cnt
FROM sc_silver.leads_pbs
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.propostas_realizadas"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
SELECT categoria_proposta, COUNT(*) cnt
FROM sc_silver.propostas_realizadas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT combustivel, COUNT(*) AS cnt
FROM sc_silver.propostas_realizadas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT tipo_cliente, COUNT(*) AS cnt
FROM sc_silver.propostas_realizadas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
WITH src AS (
  SELECT
    descricao_pintura AS raw,
    -- normalize accents + strip non-letters
    regexp_replace(
      lower(translate(coalesce(descricao_pintura,''),
        'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
        'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC'
      )),
      '[^a-z]', ''
    ) AS s
  FROM sc_silver.propostas_realizadas
)
SELECT
  CASE
    WHEN s RLIKE 'metaliz.*especial' THEN 'Metalizada Especial'
    WHEN s RLIKE 'metaliz|^metal$'   THEN 'Metalizada'        -- includes "Metal"
    WHEN s RLIKE 'solid'             THEN 'Sólida'
    ELSE NULL
  END AS pintura_norm,
  COUNT(*) AS cnt
FROM src
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT descricao_pintura, COUNT(*) AS cnt
FROM sc_silver.propostas_realizadas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT estado_do_pedido, COUNT(*) AS cnt
FROM sc_silver.propostas_realizadas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT estado_do_pedido, COUNT(*) AS cnt
FROM sc_silver.propostas_realizadas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.viaturas"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
WITH src AS (
  SELECT
    combustivel AS raw,
    -- normaliza: baixa, remove acentos, tira não-letras
    regexp_replace(
      lower(translate(coalesce(combustivel,''),
        'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
        'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
      '[^a-z]', ''
    ) AS s
  FROM sc_silver.viaturas
)
SELECT
  CASE
    WHEN s RLIKE 'plugin|plug'                         THEN 'Híbrido Plug-in'
    WHEN s RLIKE 'hibrid' AND s RLIKE '(gasoleo|diesel)' THEN 'Híbrido Gasóleo'
    WHEN (s RLIKE 'hibrid' AND s RLIKE 'gasolin')
         OR s RLIKE '(electrico|eletrico)gasolina|gasolina(electrico|eletrico)'
                                                      THEN 'Híbrido Gasolina'
    WHEN s RLIKE 'hibrid'                              THEN 'Híbrido'
    WHEN s RLIKE 'gpl' AND s RLIKE 'gasolin'           THEN 'Gasolina/GPL'
    WHEN s = 'gpl'                                     THEN 'GPL'
    WHEN s RLIKE 'electri'                             THEN 'Elétrico'
    WHEN s RLIKE 'diesel|gasoleo'                      THEN 'Gasóleo'
    WHEN s RLIKE 'gasolin'                             THEN 'Gasolina'
    WHEN s = '' OR s IN ('na','nd') OR s RLIKE 'semespecificar'
                                                      THEN NULL
    ELSE NULL
  END AS combustivel_norm,
  COUNT(*) AS cnt
FROM src
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT combustivel, COUNT(*) AS cnt
FROM sc_silver.viaturas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
WITH src AS (
  SELECT
    tire_source AS raw,
    regexp_replace(
      lower(translate(coalesce(tire_source,''),
        'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
        'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
      '[^a-z]', ''
    ) AS s
  FROM sc_silver.viaturas
)
SELECT
  CASE
    WHEN s RLIKE '(hank.*kumho|kumho.*hank|hank.*nexen|nexen.*hank|kumho.*nexen|nexen.*kumho)'
         THEN NULL                                 -- várias marcas conhecidas na mesma célula
    WHEN s RLIKE 'dunlop'                          THEN 'Michelin'
    WHEN s RLIKE 'continental'                     THEN 'Continental'
    WHEN s RLIKE 'michelin|micheline|michelinsport|^mich$|^mich' THEN 'Michelin'
    WHEN s RLIKE 'hank|han?kook'                   THEN 'Hankook'
    WHEN s RLIKE 'nexen'                           THEN 'Nexen'
    WHEN s RLIKE 'pirel'                           THEN 'Pirelli'
    WHEN s RLIKE 'kumh|kumho'                      THEN 'Kumho'
    WHEN s RLIKE 'bridgestone'                     THEN 'Bridgestone'
    WHEN s = '' OR s IN ('na','nd')                THEN NULL
    ELSE NULL
  END AS tire_source_norm,
  COUNT(*) AS cnt
FROM src
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT tire_source, COUNT(*) AS cnt
FROM sc_silver.viaturas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT codigo_local, COUNT(*) AS cnt
FROM sc_silver.viaturas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
WITH src AS (
  SELECT
    gwms_combustivel AS raw,
    -- normaliza: minúsculas, remove acentos e tudo o que não é letra
    regexp_replace(
      lower(translate(coalesce(gwms_combustivel,''),
        'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
        'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
      '[^a-z]', ''
    ) AS s
  FROM sc_silver.viaturas
)
SELECT
  CASE
    WHEN s RLIKE 'electr'                                    THEN 'Elétrico'
    WHEN s RLIKE 'diesel|dsl|tdi'                            THEN 'Gasóleo'
    WHEN s RLIKE 'liquefiedpetrolg|liquifiedpetrolg|lpg|gpl' THEN 'GPL'
    WHEN s RLIKE 'unleaded|gasolin|petrol|lead'              THEN 'Gasolina'
    ELSE NULL
  END AS combustivel_norm,
  COUNT(*) AS cnt
FROM src
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT gwms_combustivel, COUNT(*) AS cnt
FROM sc_silver.viaturas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.viaturas_demo"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
SELECT combustivel, COUNT(*) AS cnt
FROM sc_silver.viaturas_demo
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT combustivel, COUNT(*) AS cnt
FROM sc_silver.viaturas_demo
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.bd_rede_hyundai"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.campaigns"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.campanha_vouchers"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
SELECT oferta_monetaria, COUNT(*) AS cnt
FROM sc_silver.campanha_vouchers
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT oferta_percentual, COUNT(*) AS cnt
FROM sc_silver.campanha_vouchers
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.campanhas_tecnicas"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
SELECT warranty_start_date, COUNT(*) AS cnt
FROM sc_silver.campanhas_tecnicas
GROUP BY 1
ORDER BY cnt DESC;

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.contactos_importador_pbs"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
SELECT id_contactos, COUNT(*) AS cnt
FROM sc_silver.contactos_importador_pbs
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.contactos_pbs"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.contas_importador_pbs"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.contas_pbs"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.contratos_financiamento"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
SELECT capital_em_divida, COUNT(*) AS cnt
FROM sc_silver.contratos_financiamento
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT valor_final, COUNT(*) AS cnt
FROM sc_silver.contratos_financiamento
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.deals"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.historico_de_servicos"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.leads_pbs"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
SELECT lead_status, COUNT(*) AS cnt
FROM sc_silver.leads_pbs
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT consentimento, COUNT(*) AS cnt
FROM sc_silver.leads_pbs
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT concessao, COUNT(*) AS cnt
FROM sc_silver.leads_pbs
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.propostas_realizadas"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
WITH m AS (
  SELECT
    gestor_area AS raw,
    trim(
      regexp_replace(
        -- 1) remove any email
        regexp_replace(coalesce(gestor_area,''), '(?i)[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Za-z]{2,}', ''),
        -- 2) if there is a dash leftover ("Name - ..."), keep only the part before the dash
        '^\\s*(.*?)(?:\\s*[-–—].*)?$', '\\1'
      )
    ) AS cleaned
  FROM sc_silver.propostas_realizadas
  WHERE gestor_area IS NOT NULL AND regexp_like(gestor_area, '(?i)@')
)
SELECT raw, cleaned, count(*) AS cnt
FROM m
GROUP BY raw, cleaned
ORDER BY cnt DESC;


In [0]:
%sql
SELECT gestor_area, COUNT(*) AS cnt
FROM sc_silver.propostas_realizadas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT gestor_area, COUNT(*) AS cnt
FROM sc_silver.propostas_realizadas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT tylacode, COUNT(*) AS cnt
FROM sc_silver.propostas_realizadas
GROUP BY 1
ORDER BY cnt DESC;



In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from delta.tables import DeltaTable

TBL = "sc_silver.propostas_realizadas"

df = spark.table(TBL)
delta = DeltaTable.forName(spark, TBL)

def is_exact_zero_string(colname: str):
    # true only when the text is exactly "0" (ignoring surrounding spaces)
    return F.trim(F.col(colname).cast("string")) == F.lit("0")

updates = []
for f in df.schema.fields:
    c, dt = f.name, f.dataType

    # skip complex types
    if isinstance(dt, (ArrayType, MapType, StructType)):
        continue

    if isinstance(dt, (ByteType, ShortType, IntegerType, LongType, FloatType, DoubleType)):
        cond = (F.col(c) == F.lit(0).cast(dt))
    elif isinstance(dt, DecimalType):
        cond = (F.col(c) == F.lit(0).cast(dt))
    elif isinstance(dt, StringType):
        cond = is_exact_zero_string(c)   # ONLY "0" exactly
    else:
        # dates/timestamps/booleans/etc. → ignore
        continue

    updates.append((c, cond))

# (Optional) preview how many rows would be affected per column
preview = [(c, spark.table(TBL).where(cond).count()) for c, cond in updates]
display(spark.createDataFrame(preview, ["column", "rows_to_null"]).orderBy(F.desc("rows_to_null")))

# Apply: set those exact zeros to NULL, column by column
for c, cond in updates:
    delta.update(
        condition = cond,
        set       = {c: F.lit(None)}
    )


In [0]:
%sql
SELECT valor_do_apoio_pedido, COUNT(*) AS cnt
FROM sc_silver.propostas_realizadas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT s_g_p_u_, COUNT(*) AS cnt
FROM sc_silver.propostas_realizadas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT valor_do_apoio_pedido, COUNT(*) AS cnt
FROM sc_silver.propostas_realizadas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT `probability____`, COUNT(*) AS cnt
FROM sc_silver.deals
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT `n__matriculas_por_associar`, COUNT(*) AS cnt
FROM sc_silver.deals
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
SELECT `n__matriculas_associadas`, COUNT(*) AS cnt
FROM sc_silver.deals
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
%sql
/*********************************************
SELECT `n__matriculas_por_associar`, COUNT(*) AS cnt
FROM sc_silver.deals
GROUP BY 1
ORDER BY cnt DESC;

-- sanity check there are no negatives left
SELECT COUNT(*) AS negatives_left
FROM sc_silver.deals
WHERE COALESCE(
        try_cast(`n__matriculas_por_associar` AS BIGINT),
        try_cast(regexp_replace(trim(CAST(coalesce(`n__matriculas_por_associar`,'') AS STRING)),
                                '[^0-9-]', '') AS BIGINT)
      ) < 0;
*********************************************/

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.viaturas"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
-- Preview how many rows match (should be > 0)
SELECT COUNT(*) AS matches
FROM sc_silver.viaturas
WHERE regexp_like(
  CAST(coalesce(viatura_owner_name,'') AS STRING),
  '(?i)^\\s*admin(?:\\s*[-_]*\\s*hpk)?\\s*$'
);


In [0]:
%sql
SELECT viatura_owner_name, COUNT(*) AS cnt
FROM sc_silver.viaturas
GROUP BY 1
ORDER BY cnt DESC;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, MapType, StructType

tbl = "sc_silver.viaturas_demo"
TOP_N = 100        # adjust or remove .limit() below
INCLUDE_NULLS = False

df = spark.table(tbl)

# Only columns we can group on directly
simple_cols = [f.name for f in df.schema.fields
               if not isinstance(f.dataType, (ArrayType, MapType, StructType))]

for c in simple_cols:
    q = df.select(c)
    if not INCLUDE_NULLS:
        q = q.where(F.col(c).isNotNull())
    print(f"=== {tbl}.{c} ===")
    display(
        q.groupBy(F.col(c)).count()
         .orderBy(F.desc("count"))
         .limit(TOP_N)
         .withColumnRenamed(c, "value")
         .select("value", "count")
    )


In [0]:
%sql
-- Drop specific tables
DROP TABLE IF EXISTS sc_silver.funil;

DROP TABLE IF EXISTS sc_silver.campaigns_renamed;
DROP TABLE IF EXISTS sc_silver.contactos_renamed;
DROP TABLE IF EXISTS sc_silver.deals_renamed;
DROP TABLE IF EXISTS sc_silver.leads_renamed;
DROP TABLE IF EXISTS sc_silver.propostas_realizadas_renamed;

DROP TABLE IF EXISTS sc_silver.deals_with_campaigns;
DROP TABLE IF EXISTS sc_silver.leads_with_contactos;
DROP TABLE IF EXISTS sc_silver.propostas_with_deals_with_campaigns;

In [0]:
%sql
DROP TABLE IF EXISTS sc_silver.campaigns_cleaned;
DROP TABLE IF EXISTS sc_silver.contactos_cleaned;
DROP TABLE IF EXISTS sc_silver.deals_cleaned;
DROP TABLE IF EXISTS sc_silver.leads_cleaned;
DROP TABLE IF EXISTS sc_silver.propostas_realizadas_cleaned;

In [0]:
%sql
DROP TABLE IF EXISTS sc_silver.bd_rede_hyundai_cleaned;
DROP TABLE IF EXISTS sc_silver.campanha_vouchers_cleaned;
DROP TABLE IF EXISTS sc_silver.campanhas_tecnicas_cleaned;
DROP TABLE IF EXISTS sc_silver.contactos_importador_pbs_cleaned;
DROP TABLE IF EXISTS sc_silver.contratos_financiamento_cleaned;
DROP TABLE IF EXISTS sc_silver.viaturas_cleaned;
DROP TABLE IF EXISTS sc_silver.viaturas_demo_cleaned;